In [1]:
import requests
import os

os.environ["http_proxy"] = ""
os.environ["https_proxy"] = ""

DEFAULT_HOST_IP = "10.112.100.138"


def QnA(request, host_ip=DEFAULT_HOST_IP):
    url = f"http://{host_ip}:8888/v1/chatqna"

    headers = {"Content-Type": "application/json"}

    response = requests.post(url, headers=headers, json=request)
    return response


def upload_rag_file(rag_file, host_ip=DEFAULT_HOST_IP):
    url = f"http://{host_ip}:6007/v1/dataprep/ingest"

    with open(rag_file, "rb") as f:
        files = {"files": f}
        response = requests.post(url, files=files)

    return response

In [2]:
# download pdf file
# wget https://raw.githubusercontent.com/opea-project/GenAIComps/v1.1/comps/retrievers/redis/data/nke-10k-2023.pdf

# upload pdf file with dataprep
rag_file = "./nke-10k-2023.pdf"
upload_rag_file_response = upload_rag_file(rag_file)

print("Status Code:", upload_rag_file_response.status_code)
print("Response Body:", upload_rag_file_response.text)

Status Code: 200
Response Body: {"status":200,"message":"Data preparation succeeded"}


In [3]:
request = {
        "messages": "What is the revenue of Nike in 2023?",
        "stream": False
    }

QnA_response=QnA(request)

print("Status Code:", QnA_response.status_code)
print("Response Body:", QnA_response.text)

if QnA_response.status_code==200:
    result=QnA_response.json()["choices"][0]["message"]["content"]
    answer = result.split("</think>")[-1].strip()
    print(answer)

Status Code: 200
Response Body: {"id":"chatcmpl-8jfKUDBPRXgJv3LFkquZJV","object":"chat.completion","created":1744857909,"model":"chatqna","choices":[{"index":0,"message":{"role":"assistant","content":"Okay, so I need to figure out Nike's revenue for 2023. Let me look through the information provided. The search results mention NIKE, Inc. Revenues were $51.2 billion in fiscal 2023. That seems straightforward. I should make sure that's the correct figure and present it clearly.\n</think>\n\nNike's revenue in fiscal 2023 was $51.2 billion.","audio":null},"finish_reason":"stop","metadata":null}],"usage":{"prompt_tokens":0,"total_tokens":0,"completion_tokens":0}}
Nike's revenue in fiscal 2023 was $51.2 billion.
